# Reranking Hybrid Search Strategies

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser

In [3]:
loader = TextLoader("langchain.txt")
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size= 500,
    chunk_overlap = 50,
    separators = ["\n\n", "\n"," ",""]
)

docs = splitter.split_documents(documents)

docs

[Document(metadata={'source': 'langchain.txt'}, page_content='# LangChain Modules and Use Cases\n\nLangChain provides support for several main modules, each designed to address specific aspects of building applications with language models. For each module, LangChain offers examples to get started, how-to guides, reference documentation, and conceptual overviews. Below is a detailed breakdown of these modules, organized in increasing order of complexity.\n\n---\n\n## 1. Prompts'),
 Document(metadata={'source': 'langchain.txt'}, page_content='---\n\n## 1. Prompts\n\nThe **Prompts** module focuses on prompt design, management, and optimization. It includes tools and utilities for:'),
 Document(metadata={'source': 'langchain.txt'}, page_content='- **Prompt Templates**: Construct templates that accept variables, few-shot exemplars, and structured prompts for different LLM APIs.\n- **Utilities**: Support for prompt serialization, versioning, A/B testing, and guarding against prompt injectio

In [4]:
query = "How can I use Langchain to build an application with memory and tools?"

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(docs, embedding=embedding_model)
retriever = vector_store.as_retriever(search_kwargs={"k":8})

In [8]:
llm = init_chat_model("groq:openai/gpt-oss-20b")

In [ ]:
prompt = PromptTemplate.from_template("""
                            You are a helpful assistant. Your task is to rank the following documents from most to least relevant to the user's question.

                            User's question: "{question}"  

                            Documents:
                            {documents}

                            Instructions:
                            - Think about the relevance of each document to the user's question.
                            - Return a list of document indices in ranked order, starting from the most relavant.

                            Output format: comma-seperated document indices (e.g., 2,1,3,0.....)                                    
                                      """)

In [10]:
retrieved_docs = retriever.invoke(query)
retrieved_docs

[Document(id='e7337483-8ce1-4a8f-9c9b-69262f45320f', metadata={'source': 'langchain.txt'}, page_content='# LangChain Modules and Use Cases\n\nLangChain provides support for several main modules, each designed to address specific aspects of building applications with language models. For each module, LangChain offers examples to get started, how-to guides, reference documentation, and conceptual overviews. Below is a detailed breakdown of these modules, organized in increasing order of complexity.\n\n---\n\n## 1. Prompts'),
 Document(id='7c634bb8-b38a-4d2c-8965-725f65d2c9ce', metadata={'source': 'langchain.txt'}, page_content='### 8. Comparing Models\nExperimenting with different prompts, models, and chains is a key part of developing the best possible application. The ModelLaboratory makes this process easier.\n\n---\n\nLangChain provides a comprehensive suite of tools and modules to simplify and accelerate the development of applications powered by language models. By leveraging these

In [11]:
chain = prompt | llm | StrOutputParser()
chain

PromptTemplate(input_variables=['documents', 'question'], input_types={}, partial_variables={}, template='\n                            You are a helpful assistant. Your task is to rank the following documents from most to least relevant to the user\'s question.\n\n                            User\'s question: "{question}"  \n\n                            Documents:\n                            {documents}\n\n                            Instructions:\n                            - Think about the relevance of each document to the user\'s question.\n                            - Return a list of document indices in ranked order, starting from the most relavant.\n\n                            Output format: comma-seperated document indices                                     \n                                      ')
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000274EACB5FD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000

In [12]:
doc_lines = [f"{i+1}. {doc.page_content}" for i, doc in enumerate(retrieved_docs)]
formatted_docs = "\n".join(doc_lines)

formatted_docs

'1. # LangChain Modules and Use Cases\n\nLangChain provides support for several main modules, each designed to address specific aspects of building applications with language models. For each module, LangChain offers examples to get started, how-to guides, reference documentation, and conceptual overviews. Below is a detailed breakdown of these modules, organized in increasing order of complexity.\n\n---\n\n## 1. Prompts\n2. ### 8. Comparing Models\nExperimenting with different prompts, models, and chains is a key part of developing the best possible application. The ModelLaboratory makes this process easier.\n\n---\n\nLangChain provides a comprehensive suite of tools and modules to simplify and accelerate the development of applications powered by language models. By leveraging these modules, developers can build robust, scalable, and efficient solutions tailored to their specific needs.\n3. ---\n\n# Use Cases\n\nLangChain modules can be used in a variety of ways. Below are some of th

In [14]:
response = chain.invoke({"question": query, "documents": formatted_docs})

response

'6,5,3,8,1,7,4,2'

In [ ]:
# Final order the documents should be after reranking

indices = [int(x.strip())-1 for x in response.split(',') if x.strip().isnumeric()]
indices

[5, 4, 2, 7, 0, 6, 3, 1]

In [16]:
reranked_docs = [retrieved_docs[i] for i in indices if 0<=i<len(retrieved_docs)]

reranked_docs

[Document(id='5529430f-c3e1-4fda-985e-fc9235982fa8', metadata={'source': 'langchain.txt'}, page_content='- **Agent Frameworks**: Tool adapters (APIs, crawlers, calculators, databases), action schemas, and planner/executor patterns.\n- **Safety Features**: Action whitelisting, step limits, sandboxing, and observation sanitization.\n- **Examples**: Web search assistants, API-driven automation, task orchestration, and multi-tool reasoning flows.\n\n---\n\n## 8. Memory\n\nThe **Memory** module provides mechanisms for persisting and recalling conversational or task state across sessions. Key features include:'),
 Document(id='581af320-3280-4b8a-81a5-08c10ce51123', metadata={'source': 'langchain.txt'}, page_content='---\n\n## 4. Utils\n\nThe **Utils** module provides a broad collection of utilities to make LLM applications practical and composable. Examples include:'),
 Document(id='b7f15fa3-d1d8-484c-a287-068d037e8af1', metadata={'source': 'langchain.txt'}, page_content='---\n\n# Use Cases\

In [20]:
print("Final Reranked results:")
for i, doc in enumerate(reranked_docs, 1):
    print(f"\n Ranks {i}: \n {doc.page_content}")

Final Reranked results:

 Ranks 1: 
 - **Agent Frameworks**: Tool adapters (APIs, crawlers, calculators, databases), action schemas, and planner/executor patterns.
- **Safety Features**: Action whitelisting, step limits, sandboxing, and observation sanitization.
- **Examples**: Web search assistants, API-driven automation, task orchestration, and multi-tool reasoning flows.

---

## 8. Memory

The **Memory** module provides mechanisms for persisting and recalling conversational or task state across sessions. Key features include:

 Ranks 2: 
 ---

## 4. Utils

The **Utils** module provides a broad collection of utilities to make LLM applications practical and composable. Examples include:

 Ranks 3: 
 ---

# Use Cases

LangChain modules can be used in a variety of ways. Below are some of the common use cases supported by LangChain:

### 1. Agents
Agents use a language model to interact with other tools. They can be used for grounded question/answering, interacting with APIs, or taking 